# Phase 1: Core Pipeline - Educational Walkthrough

This notebook walks through the complete Phase 1 pipeline step-by-step.

**What you'll learn:**
1. How to fetch emails from Gmail API
2. How to parse HTML newsletters
3. How to extract events using LLMs
4. How to generate embeddings
5. How to store in Qdrant vector database
6. How to search semantically

**Prerequisites:**
- Phase 0 complete (environment setup)
- Gmail OAuth configured
- Qdrant running in Docker
- OpenAI API key set in `.env`

# Playground: Validate Functionalities here 
- Add Individual Scripts in the Actual Project Path 
- Run code to Understand Each File / Module


### 1. Create & Test Gmail Client (app/services/gmail_client.py) 

In [1]:
# Test Gmail client
import os
import sys
from pathlib import Path

#sys.path.insert(0, '/Users/Shared/ALL WORKSPACE/Hackathons/mom_hack/feedprism_main')
# Get the absolute path to the project root (feedprism_main)
# Assuming the notebook is in feedprism_main/notebooks/
project_root = Path.cwd().parent.absolute()

# Change working directory to project root
os.chdir(project_root)

# Add project root to Python path
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

from app.services.gmail_client import GmailClient

# Initialize client
client = GmailClient()

# Fetch 5 recent emails
emails = client.fetch_content_rich_emails(days_back=7, max_results=5)

print(f"\n✅ Fetched {len(emails)} emails")
if emails:
    first = emails[0]
    print(f"\nFirst email:")
    print(f"  Subject: {first['subject']}")
    print(f"  From: {first['from']}")
    print(f"  Has HTML: {first['body_html'] is not None}")

    second = emails[1]
    print(f"\Second email:")
    print(f"  Subject: {second['subject']}")
    print(f"  From: {second['from']}")
    print(f"  Has HTML: {second['body_html'] is not None}")

    third = emails[2]
    print(f"\Third email:")
    print(f"  Subject: {third['subject']}")
    print(f"  From: {third['from']}")
    print(f"  Has HTML: {third['body_html'] is not None}")

    fourth = emails[3]
    print(f"\Fourth email:")
    print(f"  Subject: {fourth   ['subject']}")
    print(f"  From: {fourth['from']}")
    print(f"  Has HTML: {fourth['body_html'] is not None}")

<>:35: SyntaxWarning: invalid escape sequence '\S'
<>:41: SyntaxWarning: invalid escape sequence '\T'
<>:47: SyntaxWarning: invalid escape sequence '\F'
<>:35: SyntaxWarning: invalid escape sequence '\S'
<>:41: SyntaxWarning: invalid escape sequence '\T'
<>:47: SyntaxWarning: invalid escape sequence '\F'
/var/folders/l5/vvzf291d5cx881b14hn6pc200000gn/T/ipykernel_59738/3551444189.py:35: SyntaxWarning: invalid escape sequence '\S'
  print(f"\Second email:")
/var/folders/l5/vvzf291d5cx881b14hn6pc200000gn/T/ipykernel_59738/3551444189.py:41: SyntaxWarning: invalid escape sequence '\T'
  print(f"\Third email:")
/var/folders/l5/vvzf291d5cx881b14hn6pc200000gn/T/ipykernel_59738/3551444189.py:47: SyntaxWarning: invalid escape sequence '\F'
  print(f"\Fourth email:")
2025-11-26 16:05:02.841 | INFO     | app.services.gmail_client:_authenticate:45 - Loading Gmail token from cache
2025-11-26 16:05:02.844 | SUCCESS  | app.services.gmail_client:_authenticate:67 - Gmail API authenticated successfully
2


✅ Fetched 5 emails

First email:
  Subject: Your guide to building apps with Google Antigravity
  From: Michael Spencer and Jeff Morhous from AI Supremacy  <aisupremacy+guides@substack.com>
  Has HTML: True
\Second email:
  Subject: 😺 Why is the US building an AI Manhattan Project?
  From: The Neuron <theneuron@newsletter.theneurondaily.com>
  Has HTML: True
\Third email:
  Subject: Get ready for all things tech and its future ✨
  From: upGrad <info@updates.upgrad.com>
  Has HTML: True
\Fourth email:
  Subject: Hey, we’ve prepared your MSME brief with the ₹10,00,000 cap
  From: Flexi Loan Desk <info@pool.driftem.com>
  Has HTML: True


### 2. Create & Test Email Parsing 
- (app/services/parser.py)

In [2]:
import sys
sys.path.insert(0, '/Users/Shared/ALL WORKSPACE/Hackathons/mom_hack/feedprism_main')

from app.services.parser import EmailParser

# Initialize parser
parser = EmailParser()

# Test with sample newsletter HTML
sample_html = """
<html>
<head><title>AI Weekly Newsletter</title></head>
<body>
    <h1>This Week in AI</h1>
    <p>Join us for the <strong>AI Summit 2025</strong> on December 15th.</p>
    <p><a href="https://example.com/register">Register here</a></p>
    
    <h2>Featured Article</h2>
    <p>Read about <a href="https://example.com/article">The Future of LLMs</a></p>
    
    <!-- Tracking pixel -->
    <img src="https://track.example.com/pixel.gif" width="1" height="1">
    
    <footer style="font-size:10px">
        <a href="https://example.com/unsubscribe">Unsubscribe</a>
    </footer>
</body>
</html>
"""

# Parse
result = parser.parse_html_email(sample_html)

print("✅ HTML Parser Test Results:")
print(f"\n📧 Title: {result['title']}")
print(f"\n📝 Text Length: {len(result['text'])} characters")
print(f"\n🔗 Links Found: {len(result['links'])}")

for i, link in enumerate(result['links'], 1):
    print(f"   {i}. {link['text']} → {link['url']}")

print(f"\n📄 Clean Text Preview:")
print(result['text'][:300])

# Verify tracking pixel was removed
assert 'pixel.gif' not in result['text'], "Tracking pixel should be removed"
# Verify unsubscribe footer was removed
assert 'unsubscribe' not in result['text'].lower(), "Unsubscribe footer should be removed"
# Verify links were preserved
assert len(result['links']) >= 2, "Should extract at least 2 links"

print("\n✅ All assertions passed!")

2025-11-26 16:05:10.907 | DEBUG    | app.services.parser:__init__:59 - EmailParser initialized with html2text converter
2025-11-26 16:05:10.911 | DEBUG    | app.services.parser:_clean_html:178 - HTML cleaned: removed scripts, styles, tracking pixels
2025-11-26 16:05:10.911 | DEBUG    | app.services.parser:_extract_links:260 - Extracted 2 valid links
2025-11-26 16:05:10.912 | SUCCESS  | app.services.parser:parse_html_email:108 - Parsed email: 201 chars, 2 links


✅ HTML Parser Test Results:

📧 Title: AI Weekly Newsletter

📝 Text Length: 201 characters

🔗 Links Found: 2
   1. Register here → https://example.com/register
   2. The Future of LLMs → https://example.com/article

📄 Clean Text Preview:
# This Week in AI

Join us for the **AI Summit 2025** on December 15th.

[Register here](https://example.com/register)

## Featured Article

Read about [The Future of LLMs](https://example.com/article)

✅ All assertions passed!


### 3. Create & Test 'Event' Data Model 
- app/models/extraction.py

In [6]:
import sys
sys.path.insert(0, '/Users/Shared/ALL WORKSPACE/Hackathons/mom_hack/feedprism_main')

from app.models.extraction import ExtractedEvent, EventExtractionResult, EventStatus
from datetime import datetime, timedelta

print("✅ Event Data Models Test\n")

# Test 1: Create valid event
event = ExtractedEvent(
    title="Machine Learning Workshop",
    description="Hands-on workshop covering neural networks",
    start_time="2025-12-20T14:00:00",
    location="Online",
    tags=["ML", "Workshop"]
)

print(f"1. Created event: {event.title}")
print(f"   Status: {event.compute_status()}")

# Test 2: JSON Schema generation
schema = EventExtractionResult.model_json_schema()
print(f"\n2. ✅ JSON Schema generated: {len(schema)} keys")

print("\n✅ All model tests passed!")

✅ Event Data Models Test

1. Created event: Machine Learning Workshop
   Status: EventStatus.UPCOMING

2. ✅ JSON Schema generated: 7 keys

✅ All model tests passed!


### 4. Test LLM Extraction of Events from a Given Email

In [3]:
import sys
sys.path.insert(0, '../')  # Ensure project root is in path
from app.services.extractor import LLMExtractor
async def test():
    extractor = LLMExtractor()
    
    email = """
    Join us for the AI Summit 2025!
    Date: December 15, 2025 at 2:00 PM EST
    Location: Online
    Register: https://example.com/register
    Free for students, $50 for professionals.
    """
    
    # Note: Ensure OPENAI_API_KEY is set in .env
    result = await extractor.extract_events(email, "AI Summit 2025")
    
    print(f"✅ Extracted {len(result.events)} events")
    print(f"   Confidence: {result.confidence:.2f}")
    
    if result.events:
        event = result.events[0]
        print(f"   Title: {event.title}")
        print(f"   Location: {event.location}")
# Run the test directly
await test()

2025-11-26 16:05:15.734 | INFO     | app.services.extractor:__init__:33 - LLMExtractor initialized: model=gpt-4o-mini
2025-11-26 16:05:15.735 | INFO     | app.services.extractor:extract_events:41 - Extracting events from: 'AI Summit 2025...'
2025-11-26 16:05:18.914 | SUCCESS  | app.services.extractor:extract_events:64 - Extracted 1 events (confidence: 1.00)


✅ Extracted 1 events
   Confidence: 1.00
   Title: AI Summit 2025
   Location: Online


### 5. Create & Verify Embedding Generation
- app/services/embedding_service.py

In [4]:
import sys
sys.path.insert(0, '/Users/Shared/ALL WORKSPACE/Hackathons/mom_hack/feedprism_main')

from app.services.embedder import EmbeddingService
import numpy as np

print("✅ Embedding Service Test\n")

embedder = EmbeddingService()

# Test 1: Single embedding
text = "Machine learning workshop on neural networks"
vec = embedder.embed_text(text)

print(f"1. Generated embedding:")
print(f"   Dimension: {len(vec)}")
print(f"   First 5 values: {vec[:5]}")

# Test 2: Semantic similarity
text1 = "AI conference about deep learning"
text2 = "Machine learning workshop"
text3 = "Cooking recipes for dinner"

vec1 = np.array(embedder.embed_text(text1))
vec2 = np.array(embedder.embed_text(text2))
vec3 = np.array(embedder.embed_text(text3))

# Cosine similarity
sim_12 = np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))
sim_13 = np.dot(vec1, vec3) / (np.linalg.norm(vec1) * np.linalg.norm(vec3))

print(f"\n2. Semantic similarity:")
print(f"   AI conference ↔ ML workshop: {sim_12:.3f}")
print(f"   AI conference ↔ Cooking: {sim_13:.3f}")

assert sim_12 > sim_13, "Related texts should be more similar"

# Test 3: Batch embedding
texts = ["Event 1", "Event 2", "Event 3"]
vecs = embedder.embed_batch(texts)

print(f"\n3. Batch embedding:")
print(f"   Generated {len(vecs)} vectors")

print("\n✅ All tests passed!")

/Users/Shared/ALL WORKSPACE/Hackathons/mom_hack/.venv/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
2025-11-26 16:23:58.087 | INFO     | app.services.embedder:__init__:27 - Loading embedding model: sentence-transformers/all-MiniLM-L6-v2


✅ Embedding Service Test



2025-11-26 16:24:02.107 | SUCCESS  | app.services.embedder:__init__:29 - Model loaded: 384D vectors


1. Generated embedding:
   Dimension: 384
   First 5 values: [-0.1260364055633545, -0.025976352393627167, 0.050567951053380966, -0.012599319219589233, 0.02731657400727272]


2025-11-26 16:24:05.730 | INFO     | app.services.embedder:embed_batch:71 - Embedding 3 texts in batches of 32



2. Semantic similarity:
   AI conference ↔ ML workshop: 0.636
   AI conference ↔ Cooking: 0.046

3. Batch embedding:
   Generated 3 vectors

✅ All tests passed!


### 6. Create and initialize Qdrant client
- app/database/qdrant_client.py

In [5]:
import sys
sys.path.insert(0, '/Users/Shared/ALL WORKSPACE/Hackathons/mom_hack/feedprism_main')

from app.database.qdrant_client import QdrantService
from app.services.embedder import EmbeddingService
from qdrant_client.models import PointStruct
import uuid

print("✅ Qdrant Service Test\n")

# Initialize services
qdrant = QdrantService()
embedder = EmbeddingService()

# Create collection
qdrant.create_collection(recreate=True)
print("1. ✅ Collection created")

# Create test points
texts = [
    "AI Conference on Machine Learning",
    "Python Workshop for Beginners",
    "Data Science Meetup",
    "Engineering Machine tools Learning Event"
]

points = []
for text in texts:
    vec = embedder.embed_text(text)
    point = PointStruct(
        id=str(uuid.uuid4()),
        vector=vec,
        payload={"title": text, "type": "event"}
    )
    points.append(point)

# Upsert points
qdrant.upsert_points(points)
print(f"2. ✅ Upserted {len(points)} points")

# Search
query_vec = embedder.embed_text("machine learning event")
results = qdrant.search(query_vec, limit=3)

print(f"\n3. Search results:")
for i, result in enumerate(results, 1):
    print(f"   {i}. {result['payload']['title']} (score: {result['score']:.3f})")

# Collection info
info = qdrant.get_collection_info()
print(f"\n4. Collection info:")
print(f"   Points: {info['points_count']}")
print(f"   Status: {info['status']}")

print("\n✅ All tests passed!")

2025-11-26 17:33:01.778 | INFO     | app.database.qdrant_client:__init__:41 - Connecting to Qdrant: localhost:6333
2025-11-26 17:33:01.802 | SUCCESS  | app.database.qdrant_client:__init__:43 - Qdrant client initialized
2025-11-26 17:33:01.803 | INFO     | app.services.embedder:__init__:27 - Loading embedding model: sentence-transformers/all-MiniLM-L6-v2


✅ Qdrant Service Test



2025-11-26 17:33:06.006 | SUCCESS  | app.services.embedder:__init__:29 - Model loaded: 384D vectors
2025-11-26 17:33:06.205 | INFO     | app.database.qdrant_client:create_collection:57 - Creating collection: feedprism_emails
2025-11-26 17:33:06.632 | SUCCESS  | app.database.qdrant_client:create_collection:67 - Collection created: feedprism_emails


1. ✅ Collection created


2025-11-26 17:33:06.948 | INFO     | app.database.qdrant_client:upsert_points:82 - Upserting 4 points to feedprism_emails
2025-11-26 17:33:07.153 | SUCCESS  | app.database.qdrant_client:upsert_points:89 - Upserted 4 points
2025-11-26 17:33:07.229 | INFO     | app.database.qdrant_client:search:120 - Searching feedprism_emails (limit=3)
2025-11-26 17:33:07.309 | SUCCESS  | app.database.qdrant_client:search:139 - Found 3 results


2. ✅ Upserted 4 points

3. Search results:
   1. Engineering Machine tools Learning Event (score: 0.743)
   2. AI Conference on Machine Learning (score: 0.574)
   3. Data Science Meetup (score: 0.367)

4. Collection info:
   Points: 4
   Status: green

✅ All tests passed!


## Setup: Import Libraries and Initialize

First, let's import all the modules we'll need and set up our environment.

In [6]:
# Standard library imports
import sys
import json
import os
import uuid
from datetime import datetime
from pathlib import Path

# IMPORTANT: Change working directory to project root
# This ensures credentials.json and token.json are found correctly
project_root = Path('/Users/Shared/ALL WORKSPACE/Hackathons/mom_hack/feedprism_main')
os.chdir(project_root)
print(f"📁 Working directory: {os.getcwd()}")

# Add project root to Python path
sys.path.insert(0, str(project_root))

# Third-party imports
import asyncio
from loguru import logger

# Our custom modules (we'll build these step by step)
from app.config import settings
from app.services.gmail_client import GmailClient
from app.services.parser import EmailParser
from app.services.extractor import LLMExtractor
from app.services.embedder import EmbeddingService
from app.database.qdrant_client import QdrantService
from app.models.extraction import ExtractedEvent, EventExtractionResult

# Configure logger for notebook
logger.remove()  # Remove default handler
logger.add(sys.stdout, level="INFO", format="<level>{level: <8}</level> | {message}")

print("✅ All imports successful!")
print(f"📁 Project root: {project_root}")
print(f"🔑 Credentials path: {settings.gmail_credentials_path}")
print(f"🎫 Token path: {settings.gmail_token_path}")
print(f"🔧 Config loaded: OpenAI key = {'***' + settings.openai_api_key[-4:]}")

📁 Working directory: /Users/Shared/ALL WORKSPACE/Hackathons/mom_hack/feedprism_main
✅ All imports successful!
📁 Project root: /Users/Shared/ALL WORKSPACE/Hackathons/mom_hack/feedprism_main
🔑 Credentials path: credentials.json
🎫 Token path: token.json
🔧 Config loaded: OpenAI key = ***JfoA


## Step 1: Fetch Emails from Gmail

**What's happening:**
- We connect to Gmail API using OAuth credentials
- Search for content-rich emails (newsletters, events)
- Fetch full email content including HTML body

**Key concepts:**
- Gmail API uses labels and search queries
- Emails have multipart structure (text + HTML)
- We prefer HTML for newsletters (richer content)

In [7]:
# Initialize Gmail client
# This will:
# 1. Load OAuth credentials from token.json
# 2. Refresh token if expired
# 3. Connect to Gmail API
gmail_client = GmailClient()

print("✅ Gmail client initialized")
print("\n📧 Fetching emails...")

# Fetch content-rich emails from last 7 days
# Parameters:
#   days_back: How far back to search
#   max_results: Maximum emails to fetch
emails = gmail_client.fetch_content_rich_emails(
    days_back=7,
    max_results=5  # Start with just 5 for this demo
)

print(f"\n✅ Fetched {len(emails)} emails")

# Let's examine the first email
if emails:
    first_email = emails[0]
    print("\n📬 First Email Details:")
    print(f"  Subject: {first_email['subject']}")
    print(f"  From: {first_email['from']}")
    print(f"  Date: {first_email['date']}")
    print(f"  Has HTML: {first_email['body_html'] is not None}")
    print(f"  HTML Length: {len(first_email['body_html']) if first_email['body_html'] else 0} characters")
    print(f"  Snippet: {first_email['snippet'][:100]}...")
else:
    print("⚠️ No emails found. Try adjusting search criteria.")

INFO     | Loading Gmail token from cache
INFO     | Refreshing expired Gmail token
SUCCESS  | Gmail API authenticated successfully
✅ Gmail client initialized

📧 Fetching emails...
INFO     | Fetching content-rich emails (last 7 days)
INFO     | Listing messages with query: 'newer_than:7d (unsubscribe OR newsletter OR event OR webinar OR course)'
SUCCESS  | Found 5 messages
INFO     | Fetching 5 messages
SUCCESS  | Fetched 5 messages
SUCCESS  | Successfully parsed 5 emails

✅ Fetched 5 emails

📬 First Email Details:
  Subject: Fwd: 🔮 Ilya Sutskever breaks silence on AI's future
  From: dishant ghai <dishant.ghai@gmail.com>
  Date: 2025-11-26T17:09:59+05:30
  Has HTML: True
  HTML Length: 78230 characters
  Snippet: ---------- Forwarded message --------- From: The Rundown AI &lt;news@daily.therundown.ai&gt; Date: W...


## Step 2: Parse HTML Email

**What's happening:**
- Take raw HTML from email
- Remove tracking pixels, scripts, styles
- Convert to clean text while preserving structure
- Extract all links

**Why this matters:**
- Newsletters have complex HTML with inline CSS
- Tracking pixels and ads add noise
- Clean text is better for LLM extraction
- Links contain important registration/info URLs

In [8]:
# Initialize HTML parser
# This uses BeautifulSoup4 + html2text
parser = EmailParser()

print("✅ Email parser initialized")

# Parse the first email's HTML
if emails and emails[0]['body_html']:
    print("\n🔍 Parsing HTML...")
    
    parsed_result = parser.parse_html_email(emails[0]['body_html'])
    
    print(f"\n✅ Parsing complete!")
    print(f"\n📄 Results:")
    print(f"  Title: {parsed_result['title']}")
    print(f"  Clean text length: {len(parsed_result['text'])} characters")
    print(f"  Links extracted: {len(parsed_result['links'])}")
    
    # Show first few links
    print(f"\n🔗 Sample Links:")
    for i, link in enumerate(parsed_result['links'][:3], 1):
        print(f"  {i}. {link['text'][:50]}... → {link['url'][:60]}...")
    
    # Show preview of clean text
    print(f"\n📝 Clean Text Preview (first 500 chars):")
    print("-" * 70)
    print(parsed_result['text'][:500])
    print("-" * 70)
    
    # Store for next step
    clean_text = parsed_result['text']
    email_subject = emails[0]['subject']
else:
    print("⚠️ No HTML body to parse")

✅ Email parser initialized

🔍 Parsing HTML...
SUCCESS  | Parsed email: 37643 chars, 51 links

✅ Parsing complete!

📄 Results:
  Title: 
  Clean text length: 37643 characters
  Links extracted: 51

🔗 Sample Links:
  1. Read Online... → https://elinke1c.daily.therundown.ai/ss/c/u001.dSnm3kaGd0BkN...
  2. Read Online... → https://elinke1c.daily.therundown.ai/ss/c/u001.dSnm3kaGd0BkN...
  3. Sign Up... → https://elinke1c.daily.therundown.ai/ss/c/u001.dSnm3kaGd0BkN...

📝 Clean Text Preview (first 500 chars):
----------------------------------------------------------------------
\---------- Forwarded message --------- 
From: **The Rundown AI** <[news@daily.therundown.ai](mailto:news@daily.therundown.ai)> 
Date: Wed, Nov 26, 2025 at 4:45 PM 
Subject: 🔮 Ilya Sutskever breaks silence on AI's future 
To: [dishant.ghai@gmail.com](mailto:dishant.ghai@gmail.com) <[dishant.ghai@gmail.com](mailto:dishant.ghai@gmail.com)> 

 
 

__

| | | | [ Read Online](https://elinke1c.daily.therundown.ai/ss/c/u001.

## Step 3: Extract Events with LLM

**What's happening:**
- Send clean text to OpenAI GPT-4o-mini
- Use structured output (JSON Schema) for guaranteed valid JSON
- Extract event details: title, date, location, link, etc.
- Get confidence score

**How structured output works:**
1. We define a Pydantic model (ExtractedEvent)
2. Pydantic generates JSON Schema
3. OpenAI guarantees response matches schema
4. No parsing errors or invalid JSON!

**Note:** This step requires OpenAI API key and will make an API call (~$0.0003 per email)

In [9]:
# Initialize LLM extractor
# This creates an async OpenAI client
extractor = LLMExtractor()

print("✅ LLM extractor initialized")
print(f"   Model: {extractor.model}")
print(f"   Temperature: {extractor.temperature} (0 = deterministic)")

# Extract events from the parsed text
# This is an async function, so we need to use await
print("\n🤖 Extracting events with LLM...")
print("   (This may take 2-4 seconds)")

# Run async extraction
extraction_result = await extractor.extract_events(
    email_text=clean_text,
    email_subject=email_subject
)

print(f"\n✅ Extraction complete!")
print(f"\n📊 Results:")
print(f"  Events found: {len(extraction_result.events)}")
print(f"  Confidence: {extraction_result.confidence:.2f} (0.0 - 1.0)")

# Display each extracted event
if extraction_result.events:
    print(f"\n🎉 Extracted Events:")
    print("=" * 70)
    
    for i, event in enumerate(extraction_result.events, 1):
        print(f"\nEvent {i}:")
        print(f"  📌 Title: {event.title}")
        print(f"  📝 Description: {event.description[:100] if event.description else 'N/A'}...")
        print(f"  📅 Start Time: {event.start_time or 'Not specified'}")
        print(f"  🕒 Timezone: {event.timezone or 'Not specified'}")
        print(f"  📍 Location: {event.location or 'Not specified'}")
        print(f"  🔗 Registration: {event.registration_link or 'Not specified'}")
        print(f"  💰 Cost: {event.cost or 'Not specified'}")
        print(f"  🏢 Organizer: {event.organizer or 'Not specified'}")
        print(f"  🏷️  Tags: {', '.join(event.tags) if event.tags else 'None'}")
        print(f"  ⏰ Status: {event.compute_status().value}")
    
    print("=" * 70)
else:
    print("\n⚠️ No events found in this email")

INFO     | LLMExtractor initialized: model=gpt-4o-mini
✅ LLM extractor initialized
   Model: gpt-4o-mini
   Temperature: 0.0 (0 = deterministic)

🤖 Extracting events with LLM...
   (This may take 2-4 seconds)
INFO     | Extracting events from: 'Fwd: 🔮 Ilya Sutskever breaks silence on AI's futur...'
SUCCESS  | Extracted 1 events (confidence: 0.90)

✅ Extraction complete!

📊 Results:
  Events found: 1
  Confidence: 0.90 (0.0 - 1.0)

🎉 Extracted Events:

Event 1:
  📌 Title: Ilya Sutskever breaks silence on AI's future
  📝 Description: N/A...
  📅 Start Time: 2025-11-26T16:45:00
  🕒 Timezone: Not specified
  📍 Location: Online
  🔗 Registration: https://elinke1c.daily.therundown.ai/ss/c/u001.dSnm3kaGd0BkNqLYPjeMf75St_4h4uVDlb2B1USeB_H4wpkqf8EGTk1iM5Wu0HuoFIsKQVYDu_ovIaAd89V_Nvs2jsW_lRswn84NkqTYMu8JVnw2IsjtESfB6_5xHNXcHSTXHFKTw9U75oBmgjrVqB2SZ9d-NozJirB4eyul8bEm4850lnNbtwLSr25wtfAuEAVwXueEw5LwRnEluHpMLlFQtt4Fwibr0dKzZBmuMqmxo5OKU1yaYUDw4tzKyv8Z2SlRojH7BcZtV6gpBILtJ7m2luOcCaup3Z2_t8s69rXqYATYe

## Step 4: Generate Embeddings

**What's happening:**
- Convert event text to dense vector (384 dimensions)
- Use sentence-transformers (all-MiniLM-L6-v2)
- Vectors capture semantic meaning

**Why embeddings:**
- Enable semantic search ("machine learning workshop" finds "ML training")
- Similar events have similar vectors
- Much better than keyword matching

**Model details:**
- Runs locally (no API costs)
- ~50ms per embedding
- First run downloads model (~80MB)

In [10]:
# Initialize embedding service
# First run will download the model (takes ~30 seconds)
# Subsequent runs load from cache (~2 seconds)
print("🔄 Loading embedding model...")
print("   (First time: downloads ~80MB model)")
print("   (Subsequent: loads from cache)")

embedder = EmbeddingService()

print(f"\n✅ Embedding service initialized")
print(f"   Model: {embedder.model_name}")
print(f"   Dimension: {embedder.dimension}D vectors")

# Generate embeddings for each event
if extraction_result.events:
    print(f"\n🔢 Generating embeddings for {len(extraction_result.events)} events...")
    
    event_embeddings = []
    
    for i, event in enumerate(extraction_result.events, 1):
        # Combine title and description for richer embedding
        text_to_embed = f"{event.title} {event.description or ''}"
        
        # Generate embedding
        vector = embedder.embed_text(text_to_embed)
        
        event_embeddings.append({
            'event': event,
            'vector': vector,
            'text': text_to_embed
        })
        
        print(f"  Event {i}: Generated {len(vector)}D vector")
    
    # Show example vector (first 10 dimensions)
    print(f"\n📊 Example Vector (first 10 dimensions):")
    print(f"   {event_embeddings[0]['vector'][:10]}")
    print(f"   ... (374 more dimensions)")
    
    # Demonstrate semantic similarity
    if len(event_embeddings) >= 2:
        import numpy as np
        
        vec1 = np.array(event_embeddings[0]['vector'])
        vec2 = np.array(event_embeddings[1]['vector'])
        
        # Cosine similarity
        similarity = np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))
        
        print(f"\n🔍 Semantic Similarity Example:")
        print(f"   Event 1: {event_embeddings[0]['event'].title}")
        print(f"   Event 2: {event_embeddings[1]['event'].title}")
        print(f"   Similarity: {similarity:.3f} (1.0 = identical, 0.0 = unrelated)")
else:
    print("\n⚠️ No events to embed")

🔄 Loading embedding model...
   (First time: downloads ~80MB model)
   (Subsequent: loads from cache)
INFO     | Loading embedding model: sentence-transformers/all-MiniLM-L6-v2
SUCCESS  | Model loaded: 384D vectors

✅ Embedding service initialized
   Model: sentence-transformers/all-MiniLM-L6-v2
   Dimension: 384D vectors

🔢 Generating embeddings for 1 events...
  Event 1: Generated 384D vector

📊 Example Vector (first 10 dimensions):
   [-0.03619229793548584, 0.002591446042060852, 0.05195854604244232, 0.021957270801067352, 0.042309924960136414, -0.010471072047948837, 0.01925678178668022, 0.0017113657668232918, 0.049980346113443375, -0.028640037402510643]
   ... (374 more dimensions)


## Step 5: Store in Qdrant Vector Database

**What's happening:**
- Connect to Qdrant (running in Docker)
- Create collection if doesn't exist
- Store vectors + metadata (payload)
- Build HNSW index for fast search

**Qdrant concepts:**
- **Collection**: Container for vectors (like a table)
- **Point**: Single vector + payload
- **Payload**: Metadata (title, date, location, etc.)
- **HNSW**: Fast approximate nearest neighbor search

**Why Qdrant:**
- Built for vector search
- Payload filtering ("only upcoming events")
- Fast (millisecond queries)
- Free and runs locally

In [11]:
# Initialize Qdrant client
# This connects to Docker container at localhost:6333
qdrant = QdrantService()

print("✅ Qdrant client initialized")
print(f"   Host: {qdrant.host}:{qdrant.port}")
print(f"   Collection: {qdrant.collection_name}")

# Create collection (if doesn't exist)
print("\n🗄️  Creating/verifying collection...")
qdrant.create_collection(recreate=False)  # Don't delete existing data

# Get collection info
info = qdrant.get_collection_info()
print(f"\n📊 Collection Info:")
print(f"   Name: {info['name']}")
print(f"   Points: {info['points_count']}")
print(f"   Status: {info['status']}")

# Prepare points for insertion
if event_embeddings:
    from qdrant_client.models import PointStruct
    
    print(f"\n💾 Preparing {len(event_embeddings)} points for storage...")
    
    points = []
    
    for item in event_embeddings:
        event = item['event']
        vector = item['vector']
        
        # Create point with unique ID
        point = PointStruct(
            id=str(uuid.uuid4()),  # Unique identifier
            vector=vector,  # 384D embedding
            payload={  # Metadata for filtering and display
                'title': event.title,
                'description': event.description,
                'start_time': event.start_time,
                'end_time': event.end_time,
                'timezone': event.timezone,
                'location': event.location,
                'registration_link': str(event.registration_link) if event.registration_link else None,
                'tags': event.tags,
                'organizer': event.organizer,
                'cost': event.cost,
                'status': event.compute_status().value,
                'source_email_id': emails[0]['id'],
                'source_subject': emails[0]['subject'],
                'source_from': emails[0]['from'],
                'type': 'event',
                'extracted_at': datetime.now().isoformat()
            }
        )
        
        points.append(point)
        print(f"  ✓ Prepared: {event.title}")
    
    # Insert into Qdrant
    print(f"\n📤 Upserting points to Qdrant...")
    qdrant.upsert_points(points)
    
    print(f"\n✅ Successfully stored {len(points)} events!")
    
    # Get updated collection info
    updated_info = qdrant.get_collection_info()
    print(f"\n📊 Updated Collection:")
    print(f"   Total points: {updated_info['points_count']}")
else:
    print("\n⚠️ No events to store")

INFO     | Connecting to Qdrant: localhost:6333
SUCCESS  | Qdrant client initialized
✅ Qdrant client initialized
   Host: localhost:6333
   Collection: feedprism_emails

🗄️  Creating/verifying collection...
INFO     | Collection already exists: feedprism_emails

📊 Collection Info:
   Name: feedprism_emails
   Points: 4
   Status: green

💾 Preparing 1 points for storage...
  ✓ Prepared: Ilya Sutskever breaks silence on AI's future

📤 Upserting points to Qdrant...
INFO     | Upserting 1 points to feedprism_emails
SUCCESS  | Upserted 1 points

✅ Successfully stored 1 events!

📊 Updated Collection:
   Total points: 5


## Step 6: Semantic Search

**What's happening:**
- Convert search query to vector
- Find similar vectors in Qdrant
- Return events ranked by similarity
- Apply filters (type, status, etc.)

**How it works:**
1. Query: "machine learning workshop" → vector
2. Qdrant finds nearest neighbor vectors
3. Returns events with similarity scores
4. Higher score = more relevant

**Magic of semantic search:**
- "ML training" matches "machine learning workshop"
- "AI conference" matches "artificial intelligence summit"
- Works across synonyms and related concepts

In [12]:
# Let's try some searches!
print("🔍 Semantic Search Demo\n")
print("=" * 70)

# Define search queries
search_queries = [
    "machine learning workshop",
    "AI conference",
    "data science event",
    "online webinar"
]

for query in search_queries:
    print(f"\n🔎 Query: '{query}'")
    print("-" * 70)
    
    # Generate query vector
    query_vector = embedder.embed_text(query)
    
    # Search Qdrant
    # Parameters:
    #   query_vector: The embedding of our search query
    #   limit: How many results to return
    #   filter_dict: Optional filters (e.g., only upcoming events)
    results = qdrant.search(
        query_vector=query_vector,
        limit=3,
        filter_dict={'type': 'event'}  # Only return events
    )
    
    if results:
        print(f"Found {len(results)} results:\n")
        
        for i, result in enumerate(results, 1):
            payload = result['payload']
            score = result['score']
            
            print(f"{i}. {payload['title']}")
            print(f"   📊 Relevance: {score:.3f} (0.0 - 1.0)")
            print(f"   📍 Location: {payload.get('location', 'N/A')}")
            print(f"   📅 Start: {payload.get('start_time', 'N/A')}")
            print(f"   ⏰ Status: {payload.get('status', 'N/A')}")
            print(f"   🔗 Link: {payload.get('registration_link', 'N/A')[:60]}...")
            print()
    else:
        print("No results found\n")

print("=" * 70)

🔍 Semantic Search Demo


🔎 Query: 'machine learning workshop'
----------------------------------------------------------------------
INFO     | Searching feedprism_emails (limit=3)
SUCCESS  | Found 3 results
Found 3 results:

1. AI Conference on Machine Learning
   📊 Relevance: 0.715 (0.0 - 1.0)
   📍 Location: N/A
   📅 Start: N/A
   ⏰ Status: N/A
   🔗 Link: N/A...

2. Python Workshop for Beginners
   📊 Relevance: 0.560 (0.0 - 1.0)
   📍 Location: N/A
   📅 Start: N/A
   ⏰ Status: N/A
   🔗 Link: N/A...

3. Engineering Machine tools Learning Event
   📊 Relevance: 0.535 (0.0 - 1.0)
   📍 Location: N/A
   📅 Start: N/A
   ⏰ Status: N/A
   🔗 Link: N/A...


🔎 Query: 'AI conference'
----------------------------------------------------------------------
INFO     | Searching feedprism_emails (limit=3)
SUCCESS  | Found 3 results
Found 3 results:

1. AI Conference on Machine Learning
   📊 Relevance: 0.853 (0.0 - 1.0)
   📍 Location: N/A
   📅 Start: N/A
   ⏰ Status: N/A
   🔗 Link: N/A...

2. Data Scien

## Step 7: Advanced Search with Filters

**Payload filtering:**
- Search within specific criteria
- Combine semantic search + filters
- Much more powerful than search alone

**Examples:**
- "ML events" + only upcoming
- "conferences" + only free
- "workshops" + only online

In [13]:
print("🎯 Filtered Search Demo\n")
print("=" * 70)

# Search 1: Only upcoming events
print("\n🔎 Search: 'AI event' (upcoming only)")
print("-" * 70)

query_vec = embedder.embed_text("AI event")
results = qdrant.search(
    query_vector=query_vec,
    limit=5,
    filter_dict={
        'type': 'event',
        'status': 'upcoming'  # Only future events
    }
)

print(f"Found {len(results)} upcoming events\n")
for i, r in enumerate(results, 1):
    print(f"{i}. {r['payload']['title']} (score: {r['score']:.3f})")

# Search 2: Only online events
print("\n🔎 Search: 'workshop' (online only)")
print("-" * 70)

query_vec = embedder.embed_text("workshop")
results = qdrant.search(
    query_vector=query_vec,
    limit=5,
    filter_dict={
        'type': 'event',
        'location': 'Online'  # Only virtual events
    }
)

print(f"Found {len(results)} online workshops\n")
for i, r in enumerate(results, 1):
    print(f"{i}. {r['payload']['title']} (score: {r['score']:.3f})")

print("\n" + "=" * 70)

🎯 Filtered Search Demo


🔎 Search: 'AI event' (upcoming only)
----------------------------------------------------------------------
INFO     | Searching feedprism_emails (limit=5)
SUCCESS  | Found 0 results
Found 0 upcoming events


🔎 Search: 'workshop' (online only)
----------------------------------------------------------------------
INFO     | Searching feedprism_emails (limit=5)
SUCCESS  | Found 1 results
Found 1 online workshops

1. Ilya Sutskever breaks silence on AI's future (score: 0.078)



## Summary: Complete Pipeline

**What we accomplished:**

1. ✅ **Fetched emails** from Gmail API
2. ✅ **Parsed HTML** to clean text
3. ✅ **Extracted events** using LLM structured output
4. ✅ **Generated embeddings** for semantic search
5. ✅ **Stored in Qdrant** with rich metadata
6. ✅ **Searched semantically** with filters

**Key learnings:**

- **Gmail API**: OAuth authentication, multipart emails
- **HTML Parsing**: BeautifulSoup4, tracking pixel removal
- **LLM Extraction**: Structured output guarantees valid JSON
- **Embeddings**: 384D vectors capture semantic meaning
- **Qdrant**: Vector database with payload filtering
- **Semantic Search**: Find by meaning, not just keywords

**Performance:**

- Email fetch: ~3s for 5 emails
- HTML parsing: ~50ms per email
- Event extraction: ~2-3s per email
- Embedding: ~50ms per event
- Qdrant storage: ~200ms for batch
- Search: ~100ms per query

**Next steps:**

- **Phase 2**: Add courses and blogs extraction
- **Phase 3**: Implement hybrid search (dense + sparse)
- **Phase 4**: Advanced Qdrant features (grouping, discovery)
- **Phase 5**: Deduplication and temporal reasoning

## Bonus: Process Multiple Emails

Now that you understand each step, let's process all fetched emails in a loop!

In [14]:
print("🔄 Processing All Emails\n")
print("=" * 70)

total_events = 0
all_points = []

for i, email in enumerate(emails, 1):
    print(f"\n[{i}/{len(emails)}] Processing: {email['subject'][:60]}...")
    
    try:
        # Skip if no HTML
        if not email['body_html']:
            print("  ⚠️ No HTML body, skipping")
            continue
        
        # Parse
        parsed = parser.parse_html_email(email['body_html'])
        print(f"  ✓ Parsed: {len(parsed['text'])} chars, {len(parsed['links'])} links")
        
        # Extract
        result = await extractor.extract_events(parsed['text'], email['subject'])
        print(f"  ✓ Extracted: {len(result.events)} events (confidence: {result.confidence:.2f})")
        
        if not result.events:
            continue
        
        # Embed and create points
        for event in result.events:
            text = f"{event.title} {event.description or ''}"
            vector = embedder.embed_text(text)
            
            from qdrant_client.models import PointStruct
            point = PointStruct(
                id=str(uuid.uuid4()),
                vector=vector,
                payload={
                    'title': event.title,
                    'description': event.description,
                    'start_time': event.start_time,
                    'location': event.location,
                    'registration_link': str(event.registration_link) if event.registration_link else None,
                    'tags': event.tags,
                    'status': event.compute_status().value,
                    'source_email_id': email['id'],
                    'source_subject': email['subject'],
                    'type': 'event'
                }
            )
            all_points.append(point)
        
        total_events += len(result.events)
        print(f"  ✓ Created {len(result.events)} points")
        
    except Exception as e:
        print(f"  ❌ Error: {e}")
        continue

# Store all points
if all_points:
    print(f"\n💾 Storing {len(all_points)} total events...")
    qdrant.upsert_points(all_points)
    print(f"✅ Complete!")

print(f"\n" + "=" * 70)
print(f"📊 Final Statistics:")
print(f"   Emails processed: {len(emails)}")
print(f"   Events extracted: {total_events}")
print(f"   Points stored: {len(all_points)}")

# Final collection info
final_info = qdrant.get_collection_info()
print(f"   Total in database: {final_info['points_count']} points")
print("=" * 70)

🔄 Processing All Emails


[1/5] Processing: Fwd: 🔮 Ilya Sutskever breaks silence on AI's future...
SUCCESS  | Parsed email: 37643 chars, 51 links
  ✓ Parsed: 37643 chars, 51 links
INFO     | Extracting events from: 'Fwd: 🔮 Ilya Sutskever breaks silence on AI's futur...'
SUCCESS  | Extracted 1 events (confidence: 0.90)
  ✓ Extracted: 1 events (confidence: 0.90)
  ✓ Created 1 points

[2/5] Processing: RBI Retail Direct - New RBI Retail Direct Portal...
SUCCESS  | Parsed email: 337 chars, 0 links
  ✓ Parsed: 337 chars, 0 links
INFO     | Extracting events from: 'RBI Retail Direct - New RBI Retail Direct Portal...'
SUCCESS  | Extracted 1 events (confidence: 0.90)
  ✓ Extracted: 1 events (confidence: 0.90)
  ✓ Created 1 points

[3/5] Processing: I made my resume site. It wasn’t that deep. | DEV Digest...
SUCCESS  | Parsed email: 4601 chars, 11 links
  ✓ Parsed: 4601 chars, 11 links
INFO     | Extracting events from: 'I made my resume site. It wasn’t that deep. | DEV ...'
SUCCESS  | Extracte